In [1]:
import os

In [2]:
%pwd

'c:\\git_enivornment\\Chicken-diesases\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\git_enivornment\\Chicken-diesases'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.comman import read_yaml, create_directories
import tensorflow as tf
from pathlib import Path

CONFIG_FILE_PATH = Path("config/config.yaml")
PARAMS_FILE_PATH = Path("params.yaml")

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [8]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time
from src.cnnClassifier import logger

In [9]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        logger.info(f"{tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)}")
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_ckpt_callbacks(self):
        logger.info(f"{self.config.checkpoint_model_filepath}")
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= self.config.checkpoint_model_filepath)


    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks,self._create_ckpt_callbacks]

In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

[2024-12-18 11:19:54,121: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2024-12-18 11:19:54,133: INFO: comman: yaml file: params.yaml loaded successfully]
[2024-12-18 11:19:54,137: INFO: comman: created directory at: artifacts]
[2024-12-18 11:19:54,141: INFO: comman: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2024-12-18 11:19:54,149: INFO: comman: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2024-12-18 11:19:54,200: INFO: 3825924868: <keras.src.callbacks.tensorboard.TensorBoard object at 0x000002100A556E20>]
[2024-12-18 11:19:54,202: INFO: 3825924868: artifacts\prepare_callbacks\checkpoint_dir\model.keras]
